Еще незаконченная модель, с использованием знаний про кластеризацию у каждого спутника

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = '../'
RND_SEED = 44 

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.61 s


### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

### Model training

In [5]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [6]:
def extractTimeFeats(data):
    timeFeats = data['epoch'].apply(lambda x: str(x).split('T')[1].split('.')[0].split(':')).values
    h,m,s = [],[],[]
    for feat in timeFeats:
        h.append(feat[0]); m.append(feat[1]); s.append(feat[2])
    return h,m,s

In [7]:
h,m,s = extractTimeFeats(X)
X.loc[:, 'h'] = list(map(int, h)); X.loc[:, 'm'] = list(map(int, m)); X.loc[:, 's'] =list(map(int, s))
X.drop(columns = 'epoch', inplace = True)

h,m,s = extractTimeFeats(Xtest)
Xtest.loc[:, 'h'] = list(map(int, h)); Xtest.loc[:, 'm'] = list(map(int, m)); Xtest.loc[:, 's'] = list(map(int, s))
Xtest.drop(columns = 'epoch', inplace = True)

In [85]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
model = LinearRegression()
cluster = KMeans(n_clusters=24, tol = 1e-3, algorithm = 'elkan', random_state=RND_SEED)
trainSize = 0.8
submissionValid = train.copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
submissionIndexes = []
SMP = []
smp = []
nSmapes = 6 * 300
nTargets = 6
for sat_id in Xtest['sat_id'].unique():
    if sat_id in linearSat:
        satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
        satY = y[X['sat_id'] == sat_id]
        satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])

        size = int(satX.shape[0] * trainSize)
        Xtr, Xval = satX.iloc[:size, :], satX.iloc[size:, :]
        ytr, yval = satY.iloc[:size, :], satY.iloc[size:, :]

        labelsTrain = cluster.fit_predict(Xtr)
        labelsTest = cluster.predict(Xval)
        smp = []
        for label in np.unique(labelsTest):
            for i in range(nTargets):
                model.fit(Xtr[labelsTrain == label], ytr[labelsTrain == label].iloc[:, i])
                ypred = model.predict(Xval[labelsTest == label])
                smp.append(smape(ypred, yval[labelsTest == label].iloc[:, i]))
                SMP.append(smape(ypred, yval[labelsTest == label].iloc[:, i]))
        print(f'SatId:',sat_id, 'SMAPE:', 100*(1- np.mean(smp)))
print(f'SatId:',sat_id, 'FINAL SMAPE:', 100*(1- np.mean(SMP)))

SatId: 1 SMAPE: 86.00557294311682
SatId: 2 SMAPE: 71.80048003363014
SatId: 3 SMAPE: 68.22473606469768
SatId: 4 SMAPE: 53.04999885464908
SatId: 25 SMAPE: 78.89855132835709
SatId: 27 SMAPE: 80.1710414462372
SatId: 32 SMAPE: 57.80861038673688
SatId: 38 SMAPE: 70.58404410294284
SatId: 41 SMAPE: 73.83814534500537
SatId: 45 SMAPE: 55.970501509700846
SatId: 59 SMAPE: 61.66538800664058
SatId: 63 SMAPE: 55.11540920195495
SatId: 84 SMAPE: 68.8024848336237
SatId: 86 SMAPE: 93.78852715136836
SatId: 92 SMAPE: 59.88347512720043
SatId: 98 SMAPE: 58.18124308335387
SatId: 100 SMAPE: 96.11524237286649
SatId: 101 SMAPE: 64.7346857596134
SatId: 109 SMAPE: 90.50812406280096
SatId: 112 SMAPE: 67.51399324548693
SatId: 118 SMAPE: 60.755848298496495
SatId: 123 SMAPE: 62.51334187481867
SatId: 124 SMAPE: 88.68558497971244
SatId: 125 SMAPE: 70.74560491674383
SatId: 126 SMAPE: 69.742750550588
SatId: 133 SMAPE: 68.25995022255032
SatId: 135 SMAPE: 99.87175553260126
SatId: 149 SMAPE: 65.6002248802088
SatId: 151 SMAPE

In [44]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
nTargets = 6
for sat_id in tqdm(Xtest['sat_id'].unique()):
    satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])
    
    for i in range(nTargets):
        model.fit(satX, satY.iloc[:, i])
        ypred = model.predict(satXtest)
        submission.loc[satXtest.index, submission.columns[i+1]] = ypred

  0%|                                                                                                                                                                                                                  | 0/300 [00:00<?, ?it/s]C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:12<00:00,  4.03it/s]


In [46]:
submission.to_csv('submission.csv', index = None)